In [1]:
import pandas as pd
import numpy as np
import gc
from random import randint, randrange, random, choice
from pathlib import Path
import config

In [5]:
dict1 = {
    'dict1_1': int,
    'dict1_2': float,

}

dict2 = {
    'dict2_1': int,
    'dict2_2': float,

}


print('dict1')
print(dict1)
print('dict2')
print(dict2)
dict1.update(dict2)
print('dict3')
print(dict1)

dict1
{'dict1_1': <class 'int'>, 'dict1_2': <class 'float'>}
dict2
{'dict2_1': <class 'int'>, 'dict2_2': <class 'float'>}
dict3
{'dict1_1': <class 'int'>, 'dict1_2': <class 'float'>, 'dict2_1': <class 'int'>, 'dict2_2': <class 'float'>}


In [8]:
dict1.keys()

<function dict.keys>

In [11]:
a=1

b=0

z = np.divide(1, 0, out=np.array(np.nan), where=b!=0)
float(z)

nan

In [16]:
df = pd.DataFrame(data={"user_id": [1, 2, 3, 4], "content_id": [5, 6, 7, 8]})
df[["user_id", "content_id"]]

,user_id,content_id
0,1,5
1,2,6
2,3,7
3,4,8


In [25]:
dict_user = {i: i*10 for i in df.user_id}
dict_content = {i: i*20 for i in df.user_id}
list_of_users = [1, 3]
dict_user_fromkeys = dict_user.fromkeys(list_of_users)
print("dict_user_fromkeys")
print(dict_user)
dict_user_fromkeys

dict_user_fromkeys
{1: 10, 2: 20, 3: 30, 4: 40}


{1: None, 3: None}

In [18]:
df_test = pd.DataFrame.from_records(data=[dict_user, dict_content])
df_test

,1,2,3,4
0,10,20,30,40
1,20,40,60,80


In [2]:
def preprocess_train_data(df, questions_df, target, dtypes):
    # Exclude lectures
    df = df[df[target] != -1].reset_index(drop=True, inplace=False)
    # Fill NaN values in the 'prior_question_had_explanation' columns
    df['prior_question_had_explanation'].fillna(False, inplace=True)
    # Set type
    df = df.astype(dtypes)

    # Answer for the previous questions of users
    df['lag'] = df.groupby('user_id')[target].shift()
    # For each user (groupby('user_id')), compute the cummulative number of correct answers and number answers in general
    groupby = df.groupby('user_id')['lag']
    cum = groupby.agg(['cumsum', 'cumcount'])

    # User correctness (measure the users' learning progress)
    df['user_correctness'] = cum['cumsum'] / cum['cumcount']
    # Drop the 'lag' feature
    df.drop(columns=['lag'], inplace=True)
    df.head()

    # Overall correctness of users
    user_agg = df.groupby('user_id')[target].agg(['sum', 'count'])
    # Overall difficulty of questions
    content_agg = df.groupby('content_id')[target].agg(['sum', 'count'])

    # Take only 24 last observations of each user
    # df = df.groupby('user_id').tail(24).reset_index(drop=True)
    print("df before merge")
    print(df.head(1))

    # pd.concat([df_test.reset_index(drop=True), df_user.reindex(df_test['user_id'].values).reset_index(drop=True)],
    #           axis=1)

    df_concat = pd.concat(
        [df.reset_index(drop=True), questions_df.reindex(df['user_id'].values).reset_index(drop=True)],
        axis=1, )
    df_merge = pd.merge(df, questions_df, left_on='content_id', right_on='question_id', how='left')

    print("df_concat after merge")
    print(df_concat.head(1))

    print("df_merge after merge")
    print(df_merge.head(1))

    print(" df_concat.shape, df_merge.shape", df_concat.shape, df_merge.shape)
    print("df_merge == df_concat", df_concat.shape, df_merge.shape)
    for col in df_merge:
        print(col, df_merge[col].equals(df_concat[col]))
    print(df_merge.equals(df_concat))
    df = df_concat
    df.drop(columns=['question_id'], inplace=True)

    # How many questions have been answered in each content ID?
    df['content_count'] = df['content_id'].map(content_agg['count']).astype('int32')
    # How hard are questions in each content ID?
    df['content_id'] = df['content_id'].map(content_agg['sum'] / content_agg['count'])

    return user_agg, content_agg, df

In [3]:
data_path = Path("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata")
kaggle_path = Path.cwd()/"kaggle_tmp"
questions_df = pd.read_csv(data_path/'questions.csv', usecols = [0, 3], dtype = {'question_id': 'int16', 'part': 'int8'})

train_df = pd.read_feather(data_path/"train_1e5.feather", columns=config.dtypes.keys())
train_user_agg, train_content_agg, train_df = preprocess_train_data(train_df, questions_df, config.target, config.dtypes)

In [15]:
train = pd.DataFrame(data=list(range(4)), columns=["train1"])
train

,train1
0,0
1,1
2,2
3,3


In [16]:
questions = pd.DataFrame(data=list(range(3, 7)), columns=["questions1"])

questions

,questions1
0,3
1,4
2,5
3,6


In [25]:
conc = pd.concat([train.reset_index(drop=True), questions.reindex(train.train1.values).reset_index(drop=True)], axis=1)
conc

,train1,questions1
0,0,3
1,1,4
2,2,5
3,3,6


In [26]:
merg = train.merge(questions, left_index=True,  right_index=True)
merg

,train1,questions1
0,0,3
1,1,4
2,2,5
3,3,6


In [27]:
conc.equals(merg)

True

In [6]:
a = [1, 2, 3]
b = [6, 7, 8]
c = [*a, *b]
c

[1, 2, 3, 6, 7, 8]

In [17]:
col1 = range(5)
col2 = range(5)
col3 = range(5)
col4 = range(5)
idx1 = range(5)
idx2 = range(5)

users_df = pd.DataFrame(data=[col1, col2, col3, col4, idx1, idx2]).T
users_df.columns = ["col1", "col2", "col3", "col4", "idx1", "idx2"]
# users_df.set_index(keys=["idx1", "idx2"], inplace=True)
users_df.set_index(keys=["idx1"], inplace=True)
users_df

,col_df
idx1,
0,0
1,1
2,2
3,3
4,4


In [18]:
df1 = pd.DataFrame(data=col1, columns = ["col_df1"])
# df1.columns = ["col_ser"]
df1.index.names = ["idx1"]

df1

,col_df1
idx1,
0,0
1,1
2,2
3,3
4,4


In [19]:
ser = pd.Series(data=col1, name="series_name")
# df.columns = ["col_ser"]
ser.index.names = ["idx1"]

ser

idx1
0    0
1    1
2    2
3    3
4    4
Name: series_name, dtype: int64

In [20]:
print("users_df.join(df)")
users_df.join(df)

users_df.join(df)


,col1,col2,col3,col4,idx2,col_df
idx1,,,,,,
0,0,0,0,0,0,0
1,1,1,1,1,1,1
2,2,2,2,2,2,2
3,3,3,3,3,3,3
4,4,4,4,4,4,4


In [21]:
print("users_df.join(ser)")
users_df.join(ser)

users_df.join(ser)


,col1,col2,col3,col4,idx2,series_name
idx1,,,,,,
0,0,0,0,0,0,0
1,1,1,1,1,1,1
2,2,2,2,2,2,2
3,3,3,3,3,3,3
4,4,4,4,4,4,4


In [22]:
print("users_df.join([df, df1])")
users_df.join(df, df1)

users_df.join([df, df1])


KeyError:       col_df1
idx1         
0           0
1           1
2           2
3           3
4           4

In [ ]:
to_add = pd.DataFrame(data=2, index=[2], columns=["sum_"])
print(to_add.index)
to_add

In [ ]:
print("users_df.sum_ = users_df.sum_.add(to_add.sum_, fill_value=0).astype(int)")
users_df.loc[:, "sum_"] = users_df.loc[:, "sum_"].add(to_add.sum_, fill_value=0).astype(int)
users_df

In [2]:
from collections import defaultdict
dict = defaultdict(int)

dict["user1"] += 1
dict["user3"] += 1

In [3]:
dat = pd.DataFrame.from_dict(
    dict,
    orient="index"
)

dat.head()


,0
user1,1
user3,1


In [ ]:
print("users_df.loc[users_df.count_ == 3, 'count_'] += 1")
users_df.loc[(to_add.sum_ == 2).index, "count_"] = users_df.loc[(to_add.sum_ == 2).index, "count_"].add(1, fill_value=0)
users_df

In [ ]:
users_df.count_ = users_df.count_.where(users_df.count_ == 3).add(5, fill_value=0)
users_df

In [10]:
dat = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6]]).T, columns=["zero", "one"])
for ind, (zero, one) in dat[["zero", "one"]].iterrows():
    print(ind, zero)

0 1
1 2
2 3


In [ ]:
n_users = 20000

data_ = {
    "user_id": list(range(1, int(n_users*1e2))),
    "content": list(np.full( int((n_users)*1e2 - 1), fill_value=22))
}

data = pd.DataFrame(data_)
data.set_index("user_id", inplace=True)

In [ ]:
users_ = {
    "user_id": list(range(1, n_users)),
    "content": [choice([111, 222, 333]) for rand in range(1, n_users)]

}

users = pd.DataFrame(users_).sample(frac=1)
users.set_index("user_id", inplace=True)

print("data.shape, users.shape", data.shape, users.shape)
users.content.value_counts()

In [ ]:
print("set(data.user_id) - set(users.user_id)")
len(set(data.index) - set(users.index))

In [ ]:
valid = data.sample(frac=.3, replace=False)

In [ ]:
%%time
print("Just merge")
print(len(set(valid.index).intersection(users.index)), len(valid.index), len(users.index))


_ = valid.merge(users, on="user_id", how="left")

del _
gc.collect()

In [ ]:
%%time
print("merge with right_index=True")
print(len(set(valid.index).intersection(users.index)), len(valid.index), len(users.index))

_ = valid.merge(users, on="user_id", how="left", right_index=True)

del _
gc.collect()

In [ ]:
%%time
print("merge with right_index=True and present users filtering")
print(len(set(valid.index).intersection(users.index)), len(valid.index), len(users.index))

users_loc = users.loc[users.index.isin(valid.index) ]

_ = data.merge(users_loc, on="user_id", how="left", right_index=True)

del _
gc.collect()

In [ ]:
import numpy as pd

arr = [0, 1, 2, 4, 7, 10, 11, 15, 17, 20]

print(f"90й перцентиль = {np.quantile(arr, q=.9):.1f}")